<a href="https://colab.research.google.com/github/Rizukaf-id/challenge-ai-engineer-dot/blob/main/Challenge_AI_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge AI Engineer
oleh Rizka Fiddiyansyah

## Bagian 1: Praktik Pemrograman (Python dan Google Colab)
### Soal 1.b: Aplikasi Chatbot FAQ melalui CLI/Terminal

#### import library

In [3]:
import re
import torch
from transformers import AutoTokenizer, AutoModel

#### Load faq.txt

In [1]:
!wget https://raw.githubusercontent.com/Rizukaf-id/challenge-ai-engineer-dot/refs/heads/main/faq.txt -O faq.txt

--2025-05-21 12:13:55--  https://raw.githubusercontent.com/Rizukaf-id/challenge-ai-engineer-dot/refs/heads/main/faq.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2955 (2.9K) [text/plain]
Saving to: ‘faq.txt’

faq.txt             100%[===================>]   2.89K  --.-KB/s    in 0s      

2025-05-21 12:13:55 (27.4 MB/s) - ‘faq.txt’ saved [2955/2955]



In [6]:
with open('faq.txt', 'r', encoding='utf-8') as f:
    faq_raw = f.read().strip()

faq = re.findall(
    r'Q\s*[:：]\s*(.+?)\s*A\s*[:：]\s*(.+?)(?=(?:\nQ[:：]|$))',
    faq_raw, flags=re.DOTALL | re.IGNORECASE
)
faq = [(q.strip().replace('\n', ' '), a.strip().replace('\n', ' ')) for q, a in faq_pairs if q and a]

if not faq:
    print('FAQ kosong! Pastikan format FAQ.txt: Q: ... A: ...')
    exit(1)

#### Load Model

In [5]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

#### Build App

In [7]:
def get_embedding(text):
  encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
    model_output = model(**encoded_input)

  token_embeddings = model_output.last_hidden_state
  attention_mask = encoded_input['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  sum_embeddings = torch.sum(token_embeddings * mask, 1)
  sum_mask = torch.clamp(mask.sum(1), min=1e-9)
  return (sum_embeddings / sum_mask).squeeze()


# precompute FAQ embeddings
faq_embeddings = [q for q, a in faq]
faq_embeddings = torch.stack([get_embedding(q) for q in faq_embeddings])

def ans_question(query, cossim_threshold=0.72):
  query_embedding = get_embedding(query)

  # hitung cosine similarity
  similarities = torch.nn.functional.cosine_similarity(query_embedding.unsqueeze(0), faq_embeddings)
  top_score, top_idx = torch.max(similarities, dim=0)

  top_score = top_score.item()
  top_idx = top_idx.item

  if top_score >= cossim_threshold:
    return faq[top_idx][1]
  else:
    return 'Maaf, saya tidak dapat membantu dengan pertanyaan itu.'

def chat():
  print("📚 Chatbot Kursus Coding Anak")
  print("Ketik pertanyaan anda, atau ketik 'exit' untuk keluar.\n")

  while True:
    query = input('👤 : ')
    if query.lower() in ['exit', 'quit', 'keluar']:
      break
    answer = ans_question(query)
    print('🤖 : ', answer)

#### Run APP

In [8]:
chat()

📚 Chatbot Kursus Coding Anak
Ketik pertanyaan anda, atau ketik 'exit' untuk keluar.

👤 : berapa harga kursus ini?
🤖 :  Maaf, saya tidak dapat membantu dengan pertanyaan itu.
👤 : berapa biaya untuk kursus ini?


TypeError: list indices must be integers or slices, not builtin_function_or_method

Bagian 2: Teori AI